In [ ]:
import serial
from time import sleep
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
from timeit import default_timer as timer
%matplotlib inline
from IPython.display import clear_output
import tensorflow as tf

In [ ]:
ser = serial.Serial()
ser.baudrate = 115200
ser.port = 'COM14'
ser.open()

In [ ]:
# a - length of the link (distance from zi-1 to zi measured along xi)
# alpha - twist of the link (angle from z-i-1 to zi measured about xi)
# d - link offset for prismatic links (can change) distance from xi-1 measured along zi-1
# theta (aka q)- is the instantaneous angle for revolute links, from xi-1 to xi measured along zi-1




class MyRobot:
    
    def DH_A(self, a, alpha, d, theta):
        return np.array([[np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
                     [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
                     [0, np.sin(alpha), np.cos(alpha), d],
                     [0,0,0,1]])
    
    
    
    def endpoint(self, in_q1 , in_q2, in_q3, in_q4, start_at=0, stop_at=None):
        if stop_at is None:
            stop_at = 5
    
        params = self.DH_params
        params[0,3] =in_q1
        params[1,3] =in_q2
        params[2,3] =in_q3
        params[3,3] =in_q4
    
        A = self.DH_A(params[start_at,0], params[start_at,1], params[start_at,2], params[start_at,3])
        for i in range(start_at+1, stop_at):
            A = np.dot(A, self.DH_A(params[i,0], params[i,1], params[i,2], params[i,3]))
        return A
    
    
    def set_angles_rads(self, q1, q2, q3, q4):
        self.set_angles([ -30+180-180*q2/np.pi, -7+90+180*q3/np.pi  , 90+180*q4/np.pi, 180*q1/np.pi ])
    
    def set_angles(self, angles):
        if self.serial is None:
            return
        
        angles = np.clip(angles, 0, 180).astype(np.uint8)
        self.angles = angles
        self.serial.write(bytearray(angles))
        self.serial.flush()
        sleep(0.001)
#        print('New angles: {}'.format(angles))


    def linear_to_rads(self, q, rate=30):
        self.linear_to( [-30+180-180*q[1]/np.pi, -7+90+180*q[2]/np.pi  , 90+180*q[3]/np.pi, 180*q[0]/np.pi], rate_ang_sec=rate )

    def linear_to_exp(self, target_angles, rate_ang_sec=60, dt=0.001):
        start_angles = self.angles
        diff = target_angles - start_angles
        maxdiff = np.absolute(diff).max()

        total_time = maxdiff/rate_ang_sec
        
        timesteps = np.linspace(0, total_time, total_time/dt)       
        velocity = [np.sin( (np.pi*timesteps/total_time) ) for i in range(0,4)]
        pos = [ start_angles[i] + diff[i]*np.cumsum(velocity[i]*timesteps)/maxdiff for i in range(0,4) ]
        
        if 0:
            plt.subplot(1,2,1)
            plt.plot(timesteps, velocity[0], 'r-')
            plt.plot(timesteps, velocity[1], 'g-')
            plt.plot(timesteps, velocity[2], 'b-')
            plt.subplot(1,2,2)
            plt.plot(timesteps, pos[0], 'r--')
            plt.plot(timesteps, pos[1], 'g--')
            plt.plot(timesteps, pos[2], 'b--')
            plt.grid()
            plt.show()
        
        for i,t in enumerate(timesteps):
            self.set_angles(  [pos[0][i], pos[1][i], pos[2][i], pos[3][i]] )        
        
        
    def linear_to(self, target_angles, rate_ang_sec=60, dt=0.001):
        start_angles = self.angles
        diff = target_angles - start_angles
        total_time = np.absolute(diff).max()/rate_ang_sec
        
        
        
        if total_time > 0:
            steps = diff/total_time

            t=0
            while t<total_time:
                self.set_angles( start_angles + steps*t )
                t = t+dt

    def home(self):
        self.set_angles(self.home_angles)
    
    def __init__(self, ser = None):
        self.serial = ser
        self.home_angles = [60, 83, 73, 98];
        self.angles = self.home_angles
        
        self.DH_params = np.array([[ 0, np.pi/2, 54.47, 0],
             [50.22, np.pi, 0,0],
            [49.15, 0, 0, 0],
            [34.22, -np.pi, 0,0],
            [-82.94, 0,0,np.pi/2]])
        print(self.DH_params.shape)
        
        self.home()

    
robot = MyRobot(ser)

sleep(0.5)

In [ ]:
def sin(alpha):
    return np.sin(alpha)

def cos(alpha):
    return np.cos(alpha)

def draw_vectors( p0, A, major=0, minor=1, l=10 ):
    v1 = A[0:3,0]
    v2 = A[0:3, 1]
    v3 = A[0:3, 2]
    v1 = v1/np.linalg.norm(v1)
    v2 = v2/np.linalg.norm(v2)
    v3 = v3/np.linalg.norm(v3)
    
    plt.plot( [p0[major], p0[major]+l*v1[major]],[p0[minor], p0[minor]+l*v1[minor]], 'r-' )
    plt.plot( [p0[major], p0[major]+l*v2[major]],[p0[minor], p0[minor]+l*v2[minor]], 'g-' )
    plt.plot( [p0[major], p0[major]+l*v3[major]],[p0[minor], p0[minor]+l*v3[minor]], 'b-' )

def plotview(P, major=0, minor=1, limits=[-100,100,-100,100], A=None):
    for i in range(0, P.shape[1]):
        plt.plot( P[major,i], P[minor,i], 'x' )
        plt.text( P[major,i], P[minor,i], 'P{}'.format(i+1) )
        if i>0:
            plt.plot( [ P[major,i-1], P[major,i]], [P[minor,i-1], P[minor,i]], 'b-x' )
    plt.plot( [ 0, P[major,0]], [0, P[minor,0]], 'b-x' )
            
    if A is not None:
        draw_vectors( P[:,i], A, major=major, minor=minor )
            
    if limits is not None:
        plt.axis(limits)
    plt.grid()

            



def draw_situation(q1, q2, q3, q4, figsize=(7,7), plot_major=True, plot_minor=True, call_show=True, target=None):
    A1 = robot.endpoint(q1, q2, q3, q4, 0,1)
    A2 =  robot.endpoint(q1, q2, q3, q4, 0,2)
    A3 =  robot.endpoint(q1, q2, q3, q4, 0,3)
    A4 =  robot.endpoint(q1, q2, q3, q4, 0,4)
    A5 =  robot.endpoint(q1, q2, q3, q4, 0,5)
    A = robot.endpoint(q1, q2, q3, q4)
    p = np.array([[0,0,0,1]]).transpose()

    p1 = np.dot(A1, p)
    p2 = np.dot(A2, p)
    p3 = np.dot(A3, p)
    p4 = np.dot(A4, p)
    p5 = np.dot(A, p)

    P = np.concatenate( (p1,p2,p3,p4,p5), axis=1)

    
    if plot_major and plot_minor:
        plt.figure(1, figsize=figsize)
        plt.subplot(2,2,1)
    if plot_major:
        plotview(P, 0,1, limits=None, A=A)
        plt.xlabel('X')
        plt.ylabel('Y')
        if target is not None:
            plt.plot(target[0], target[1], 'rx');
        plt.axis('equal')
        plt.title('Top view')

    if plot_major and plot_minor:
        plt.subplot(2,2,2)
    if plot_minor:
        plotview(P, 1,2, limits=None, A=A)
        if target is not None:
            plt.plot(target[1], target[2], 'rx');
        plt.xlabel('Y')
        plt.ylabel('Z')
        plt.axis('equal')
        plt.title('View from the left hand side')
    if call_show:
        plt.show()




In [ ]:
solution = dict()
depth = 64
stddev = 0.01
solution['fc1w'] = np.random.normal(scale=stddev, size=(7,depth)).astype(np.float32)
solution['fc1b'] = np.zeros((depth,)).astype(np.float32)
solution['fc2w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc2b'] = np.zeros((depth,)).astype(np.float32)
solution['fc3w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc3b'] = np.zeros((depth,)).astype(np.float32)
solution['fc4w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc4b'] = np.zeros((depth,)).astype(np.float32)
solution['fc5w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc5b'] = np.zeros((depth,)).astype(np.float32)
solution['fc6w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc6b'] = np.zeros((depth,)).astype(np.float32)
solution['fc7w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc7b'] = np.zeros((depth,)).astype(np.float32)
solution['fc8w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc8b'] = np.zeros((depth,)).astype(np.float32)
solution['fc9w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc9b'] = np.zeros((depth,)).astype(np.float32)

solution['predw'] = np.random.normal(scale=stddev, size=(depth,8)).astype(np.float32)
solution['predb'] = np.zeros((8,)).astype(np.float32)



In [ ]:
def generate_batch3(batch_size):
    x = np.random.uniform(-100,100, (batch_size,1))
    y = np.random.uniform(0,100, (batch_size,1))
    z = np.random.uniform(-10,100, (batch_size,1))
    xyz = np.concatenate([x,y,z], axis=1)
    xyz_rev = np.zeros((batch_size,3))
    qs = np.zeros((batch_size,4))
    vecs = np.zeros((batch_size, 4,2))
    for i in range(0, batch_size):
        qs[i,:] = solve_num(xyz[i,:], rmse_th=1e-2,  maxit=1500,step_size=0.5, epsilon=np.pi/1800)
        vecs[i, :,0] = np.cos(qs[i])
        vecs[i, :,1] = np.sin(qs[i])
        #xyz_rev[i,:] = robot.endpoint(qs[i,0], qs[i,1], qs[i,2], qs[i,3])[0:3,-1]
    return qs, xyz, vecs#, xyz_rev

qs, xyz, vecs= generate_batch3(1)

print(qs)
print(xyz)
print(vecs)

print( np.arctan2( vecs[0,:,1], vecs[0,:,0] ) )

In [ ]:




variables = dict()
for key in solution:
    variables[key] = tf.Variable(solution[key], dtype=tf.float32, name=key)

def prediction(xyz_in, keep_prob):
    fc1 = tf.matmul( xyz_in, variables['fc1w']  ) + variables['fc1b']
    fc1_act = tf.nn.tanh(fc1)
    fc1_drop = tf.nn.dropout(fc1_act, keep_prob)

    fc2 = tf.matmul( fc1_drop, variables['fc2w']  ) + variables['fc2b']
    fc2_act = tf.nn.tanh(fc2)
    fc2_drop = tf.nn.dropout(fc2_act, keep_prob)

    fc3 = tf.matmul(fc2_drop, variables['fc3w']  ) + variables['fc3b']
    fc3_act = tf.nn.tanh(fc3)
    fc3_drop = tf.nn.dropout(fc3_act, keep_prob)

    fc4 = tf.matmul( fc3_drop, variables['fc4w']  ) + variables['fc4b']
    fc4_act = tf.nn.tanh(fc4)
    fc4_drop = tf.nn.dropout(fc4_act, keep_prob)

    fc5 = tf.matmul( fc4_drop, variables['fc5w']  ) + variables['fc5b']
    fc5_act = tf.nn.tanh(fc5)
    fc5_drop = tf.nn.dropout(fc5_act, keep_prob)

    fc6 = tf.matmul( fc5_drop, variables['fc6w']  ) + variables['fc6b']
    fc6_act = tf.nn.tanh(fc6)
    fc6_drop = tf.nn.dropout(fc6_act, keep_prob)

    fc7 = tf.matmul( fc6_drop, variables['fc7w']  ) + variables['fc7b']
    fc7_act = tf.nn.tanh(fc7)
    fc7_drop = tf.nn.dropout(fc7_act, keep_prob)

    fc8 = tf.matmul( fc7_drop, variables['fc8w']  ) + variables['fc8b']
    fc8_act = tf.nn.tanh(fc8)
    fc8_drop = tf.nn.dropout(fc8_act, keep_prob)

    fc9 = tf.matmul( fc8_drop, variables['fc9w']  ) + variables['fc9b']
    fc9_act = tf.nn.tanh(fc9)
    fc9_drop = tf.nn.dropout(fc9_act, keep_prob)


    pred = tf.matmul( fc6_drop, variables['predw']  ) + variables['predb']
    pred = tf.reshape(pred, (4,2))
    
    return pred

def tf_T(a, alpha, d, theta, name=None):
    return [[tf.cos(theta), -tf.sin(theta)*tf.cos(alpha), tf.sin(theta)*tf.sin(alpha), a*tf.cos(theta)],
                       [tf.sin(theta), tf.cos(theta)*tf.cos(alpha), -tf.cos(theta)*tf.sin(alpha), a*tf.sin(theta)],
                       [0.0, tf.sin(alpha), tf.cos(alpha), d],
                       [0.0,0.0,0.0,1.0]]


def tf_A(q):
    A0 = tf_T(0.0, np.pi/2, 54.47, q[0])
    A1 = tf_T(50.22, np.pi, 0.0, q[1])
    A2 = tf_T(49.15,0.0, 0.0, q[2])
    A3 = tf_T( 34.22, -np.pi, 0.0, q[3] )
    A4 = tf_T(-82.94, 0.0, 0.0, np.pi/2)
    A = tf.matmul(A0, tf.matmul(A1, tf.matmul(A2, tf.matmul(A3, A4))))
    return A


keep_prob = tf.placeholder(tf.float32)
xyz_in = tf.placeholder(tf.float32, shape=(1, 3))
q_in = tf.placeholder(tf.float32, shape=(1,4))
q0_in = tf.placeholder(tf.float32, shape=(1,4))

q_pred_arg = tf.concat([xyz_in, q0_in], axis=1)

vecs_pred = prediction(q_pred_arg, keep_prob)
vecs_norm = tf.sqrt( tf.pow(vecs_pred[:,0],2) + tf.pow(vecs_pred[:,1],2) )
loss_vecs_norm = tf.reduce_mean( tf.pow(vecs_norm-1,2)) 



solver_vecs = tf.train.AdamOptimizer(learning_rate=0.0001)
train_op_vecs = solver_vecs.minimize(loss_vecs_norm)

q_pred = tf.atan2(vecs_pred[:,1], vecs_pred[:,0])


net_A = tf_A(q_pred)
xyz_out = net_A[0:3, 3]
q0 = np.array([[np.pi/2, np.pi/2, np.pi/3, 0]]).astype(np.float32)

w1 = 5e-4
loss_xyz = tf.reduce_mean( tf.pow(xyz_in-xyz_out,2) ) 
loss_q = tf.reduce_mean(tf.pow( q_in-q_pred ,2))

solver_xyz = tf.train.AdamOptimizer(learning_rate=0.0001)
train_op_xyz = solver.minimize(loss_xyz)

solver_q = tf.train.AdamOptimizer(learning_rate=0.0001)
train_op_q = solver.minimize(loss_q)




In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    it = 0
    progress = []
    while 1:
        
            
        qs, xyz, vecs= generate_batch3(1)

        # TRAINING THE NORMS FIRST
        _, loss_vecs_norm_cur, vecs_out, vecs_norms_out = sess.run([train_op_vecs, loss_vecs_norm, vecs_pred, vecs_norm], feed_dict={xyz_in: xyz, q0_in:q0, q_in: qs, keep_prob: 0.5})
        if it % 100 == 0:
            clear_output()
            print(vecs_out)
            print(vecs_norms_out)
            print(loss_vecs_norm)
        
        _,_, q_pred_cur, vecs_pred_cur, loss_xyz_cur, loss_q_cur = sess.run([train_op_xyz, train_op_q, q_pred, vecs_pred, loss_xyz, loss_q], feed_dict={ xyz_in: xyz, q0_in:q0, q_in: qs, keep_prob: 0.5 })
        progress.append([it, loss_xyz_cur, loss_q_cur, loss_vecs_norm_cur])
        if it % 20 == 0:
            clear_output()
            print('It: {}, loss_xyz: {}, loss_qs: {}'.format(it, loss_xyz_cur, loss_q_cur))

            print('Vectors: {}'.format(vecs_pred_cur))

            p = np.array(progress)
            for i in range(0,3):
                plt.subplot(1,3,i+1)
                plt.plot( p[np.max([0,it-1000]):it,0], p[np.max([0,it-1000]):it,i+1] )
                plt.grid()
                plt.xlabel('Iterations')
                plt.ylabel('Loss')
            plt.show()

            plt.subplot(1,2,1)
            draw_situation(qs[0,0], qs[0,1], qs[0,2], qs[0,3], target=xyz[0,:], plot_major=False, call_show=False)
            plt.subplot(1,2,2)
            draw_situation(q_pred_cur[0], q_pred_cur[1], q_pred_cur[2], q_pred_cur[3], target=xyz[0,:], plot_major=False, call_show=False)
            plt.show()
        
        for key in variables:
            solution[key] = variables[key].eval()

        it = it+1

In [ ]:
def jacobian(q, epsilon=0.0001, l=0.0001):
    J_q2 = ((robot.endpoint(q[0], q[1]+epsilon, q[2], q[3]) - robot.endpoint(q[0], q[1]-epsilon, q[2], q[3]))/(2*epsilon))
    J_q3 = ((robot.endpoint(q[0], q[1], q[2]+epsilon, q[3]) - robot.endpoint(q[0], q[1], q[2]-epsilon, q[3]))/(2*epsilon))
    J_q4 = ((robot.endpoint(q[0], q[1], q[2], q[3]+epsilon) - robot.endpoint(q[0], q[1], q[2], q[3]-epsilon))/(2*epsilon))
    J = np.stack((J_q2[0:3,-1], J_q3[0:3,-1], J_q4[0:3,-1]), axis=0)
    J = J.transpose()
    return J

def solve_num(target, q=None, q_limits=None, maxit=5000, step_size=0.1, l=100, epsilon=0.0001, rmse_th=1e-4, verbose=False):
    if q is None:
        q = [np.pi/2, np.pi/2, np.pi/2, 0]
    q = np.array(q)

    r = math.sqrt( target[0]*target[0] + target[1]*target[1] )
    q[0] = math.atan2(target[1], target[0])

    
    for it in range(0,maxit):
        endpoint = robot.endpoint(q[0], q[1], q[2], q[3])
        residual = target-endpoint[0:3,-1]
        
        MSE = np.power(residual, 2).mean()
        RMSE = np.sqrt(np.power(residual, 2).mean()) 
        if RMSE < rmse_th:
            if verbose:
                print('it: {}, MSE: {}, RMSE: {}'.format(it, MSE, RMSE))
            break

        
        J = jacobian(q, l=l, epsilon=epsilon)
        
        
        
        beta_update = np.dot(np.dot(np.linalg.inv( np.dot(J.transpose(), J) +l*np.eye(3)), J.transpose()), residual)
        q[1:]+= beta_update*step_size 
        
        if q_limits is not None:
            for l in range(1,4):
                q[l] = np.clip(q[l], q_limits[l,0], q_limits[l,1])

    return q



In [ ]:
def generate_batch(batch_size):
    # GENERATE TRANING EXAMPLES
    qs = np.random.uniform(0, np.pi/2, (batch_size, 4))
        
    xyz = np.zeros((batch_size, 3))
    for i in range(0, batch_size):
        xyz[i,:] = robot.endpoint(qs[i,0], qs[i,1], qs[i,2], qs[i,3])[0:3,-1]

    return qs, xyz

def generate_batch2(batch_size):
    x = np.random.uniform(-100,100, (batch_size,1))
    y = np.random.uniform(0,100, (batch_size,1))
    z = np.random.uniform(-10,100, (batch_size,1))
    xyz_start = np.concatenate([x,y,z], axis=1)

    xyz_stop = xyz_start + np.random.normal(0, 10, (batch_size,3))
    
    qs_start = np.zeros((batch_size,4))
    qs_stop = np.zeros((batch_size,4))
    vecs = np.zeros((batch_size, 4,2))
    for i in range(0, batch_size):
        
        qs_start[i,:] = solve_num(xyz_start[i,:], rmse_th=1e-2,  maxit=1500,step_size=0.5, epsilon=np.pi/1800)
        qs_stop[i,:] = solve_num(xyz_stop[i,:], q=qs_start[i,:], rmse_th=1e-2,  maxit=1500,step_size=0.5, epsilon=np.pi/1800)

    start = np.concatenate([xyz_start, qs_start, xyz_stop], axis=1)
    dq = qs_stop-qs_start

    
#    print('Sins: {}'.format(np.sin(dq)))
    for i in range(0, batch_size):
        vecs[i,:,0] = np.sin(qs_stop[i,:]) - np.sin(qs_start[i,:])
        vecs[i,:,1] = np.cos(qs_stop[i,:]) - np.cos(qs_start[i,:])
#    vecs[:,0] = np.sin(dq)
#    vecs[:,1] = np.cos(dq)
    
    return vecs, start
        
vecs, start = generate_batch2(1)

print('from: {}'.format( start))
print('dq: {}'.format( vecs))


In [ ]:
solution = dict()
depth = 8
stddev = 0.01
solution['fc1w'] = np.random.normal(scale=stddev, size=(10,depth)).astype(np.float32)
solution['fc1b'] = np.zeros((depth,)).astype(np.float32)
solution['fc2w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc2b'] = np.zeros((depth,)).astype(np.float32)
solution['fc3w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc3b'] = np.zeros((depth,)).astype(np.float32)
solution['fc4w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc4b'] = np.zeros((depth,)).astype(np.float32)
solution['fc5w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc5b'] = np.zeros((depth,)).astype(np.float32)
solution['fc6w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc6b'] = np.zeros((depth,)).astype(np.float32)
solution['fc7w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc7b'] = np.zeros((depth,)).astype(np.float32)
solution['fc8w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc8b'] = np.zeros((depth,)).astype(np.float32)
solution['fc9w'] = np.random.normal(scale=stddev, size=(depth,depth)).astype(np.float32)
solution['fc9b'] = np.zeros((depth,)).astype(np.float32)

solution['predw'] = np.random.normal(scale=stddev, size=(depth,8)).astype(np.float32)
solution['predb'] = np.zeros((8,)).astype(np.float32)



In [ ]:


import tensorflow as tf

xyz_in = tf.placeholder(tf.float32, (None,10))
thetas_in = tf.placeholder(tf.float32, (None,4,2))
keep_prob = tf.placeholder(tf.float32)

variables = dict()
for key in solution:
    variables[key] = tf.Variable(solution[key], dtype=tf.float32, name=key)


fc1 = tf.matmul( xyz_in, variables['fc1w']  ) + variables['fc1b']
fc1_act = tf.nn.tanh(fc1)
fc1_drop = tf.nn.dropout(fc1_act, keep_prob)

fc2 = tf.matmul( fc1_drop, variables['fc2w']  ) + variables['fc2b']
fc2_act = tf.nn.tanh(fc2)
fc2_drop = tf.nn.dropout(fc2_act, keep_prob)

fc3 = tf.matmul(fc2_drop, variables['fc3w']  ) + variables['fc3b']
fc3_act = tf.nn.tanh(fc3)
fc3_drop = tf.nn.dropout(fc3_act, keep_prob)

fc4 = tf.matmul( fc3_drop, variables['fc4w']  ) + variables['fc4b']
fc4_act = tf.nn.tanh(fc4)
fc4_drop = tf.nn.dropout(fc4_act, keep_prob)

fc5 = tf.matmul( fc4_drop, variables['fc5w']  ) + variables['fc5b']
fc5_act = tf.nn.tanh(fc5)
fc5_drop = tf.nn.dropout(fc5_act, keep_prob)

fc6 = tf.matmul( fc5_drop, variables['fc6w']  ) + variables['fc6b']
fc6_act = tf.nn.tanh(fc6)
fc6_drop = tf.nn.dropout(fc6_act, keep_prob)

fc7 = tf.matmul( fc6_drop, variables['fc7w']  ) + variables['fc7b']
fc7_act = tf.nn.tanh(fc7)
fc7_drop = tf.nn.dropout(fc7_act, keep_prob)

fc8 = tf.matmul( fc7_drop, variables['fc8w']  ) + variables['fc8b']
fc8_act = tf.nn.tanh(fc8)
fc8_drop = tf.nn.dropout(fc8_act, keep_prob)

fc9 = tf.matmul( fc8_drop, variables['fc9w']  ) + variables['fc9b']
fc9_act = tf.nn.tanh(fc9)
fc9_drop = tf.nn.dropout(fc9_act, keep_prob)


pred = tf.reshape(tf.matmul( fc4_drop, variables['predw']  ) + variables['predb'], (-1, 4,2))


loss = tf.reduce_mean(tf.pow(pred-thetas_in,2))
solver = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = solver.minimize(loss)
batch_size = 128
progress = []
it = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    while 1:
        

        qs, xyz = generate_batch2(batch_size)
        
        
        
        _, loss_cur, out = sess.run([ train_op, loss, pred ], feed_dict={ xyz_in: xyz, thetas_in: qs, keep_prob: 0.5})
        progress.append([it, loss_cur])
        
        

        
        # SOME VIZ
        if it % 2 == 0:
            clear_output()
            print('It: {}, loss: {}\n{}'.format(it, loss_cur, out.shape))

            p = np.array(progress)
            plt.plot(p[ np.max([it-1000,0]) :it,0], p[np.max([it-1000,0]):it,1], 'b-')
            plt.grid()
            plt.xlabel('Iterations')
            plt.ylabel('Loss')
            plt.show()
            
            for key in variables:
                solution[key] = variables[key].eval()

                
            q = [np.pi/2, 4.5*np.pi/5, np.pi/3, 0]
            xyz_start = robot.endpoint(q[0], q[1], q[2], q[3])[0:3,-1]
            xyz_stop = xyz_start + np.random.normal(0, 50, size=(3,))
            net_arg = np.reshape(np.concatenate((xyz_start, q, xyz_stop), axis=0), (1,10))                
            vec = sess.run(pred, feed_dict={xyz_in: net_arg, keep_prob: 1.0})
            thetas_plus = np.arctan2( vec[0,:,1], vec[0,:,0] )
            thetas_out = q+thetas_plus
            xyz_rev = robot.endpoint(thetas_out[0], thetas_out[1], thetas_out[2], thetas_out[3])[0:3,-1]
            
            
            print('Start: {}\n Stop: {}\nRev:{}\nthetas_orig:{}\nupdates:{}\nvectors:{}'.format(xyz_start, xyz_stop, xyz_rev, q, thetas_plus, vec))
            robot.set_angles_rads(thetas_out[0], thetas_out[1], thetas_out[2], thetas_out[3])
            
        
        it = it+1

In [ ]:
def linear_to(robot, target_angles, rate_ang_sec=30, dt=0.001):
        start_angles = robot.angles
        diff = target_angles - start_angles
        maxdiff = np.absolute(diff).max()

        total_time = maxdiff/rate_ang_sec
        
        timesteps = np.linspace(0, total_time, total_time/dt)       
        n = len(timesteps)
        print(n)
        velocity = [np.absolute(diff[i])*rate_ang_sec*np.sin( (np.pi*timesteps/total_time))/maxdiff  for i in range(0,4)]
#        pos = [ start_angles[i] + diff[i]*np.cumsum(velocity[i]*timesteps)/(np.absolute(diff[i])) for i in range(0,4) ]
        pos = [ np.multiply((1/total_time), cos(np.pi*timesteps/total_time)) for i in range(0,4)]
        
        plt.subplot(1,2,1)
        plt.plot(timesteps, velocity[0], 'r-')
        plt.plot(timesteps, velocity[1], 'g-')
        plt.plot(timesteps, velocity[2], 'b-')
        plt.subplot(1,2,2)
        plt.plot(timesteps, pos[0], 'r--')
        plt.plot(timesteps, pos[1], 'g--')
        plt.plot(timesteps, pos[2], 'b--')
        plt.grid()
        plt.show()
        
        #for i,t in enumerate(timesteps):
          #  robot.set_angles(  [pos[0][i], pos[1][i], pos[2][i], pos[3][i]] )
        
robot.set_angles([60, 83, 73, 98])
sleep(1)
linear_to(robot, [10, 83, 73, 98]  )

In [ ]:
draw_situation(q[0], q[1], q[2], q[3])
robot.linear_to_rads(q, rate=90)

In [ ]:

def solve_cosine(a, b, c):
    cos_c = (a*a + b*b - c*c)/(2*a*b)
    sin_c_arg = 1-cos_c*cos_c
    if(sin_c_arg < 0):
        sin_c_arg = 0
    sin_c = math.sqrt(sin_c_arg)
    return math.atan2(sin_c, cos_c), math.atan2(-sin_c, cos_c)



def solve_ccd(target, q, q_limits=None, maxit=4, rmse_th=0.1, verbose=True):
    q = np.array(q)
    q0 = q

    target_planar = np.array([ math.sqrt( target[0]*target[0] + target[1]*target[1] ), target[2]])

    # Set the obvious angle
    q[0] = math.atan2(target[1], target[0])

    for it in range(0, maxit):
        for l in range(3,0,-1):
            end_xyz = robot.endpoint(q[0], q[1], q[2], q[3])[0:3,-1]
            
            RMSE = np.linalg.norm( target-end_xyz )
            if RMSE < rmse_th:
                draw_situation(q[0], q[1], q[2], q[3], target)
                return q
            #print('RMSE: {}'.format(RMSE))
            
            link_xyz = robot.endpoint(q[0], q[1], q[2], q[3],0,l)[0:3,-1]
            end_planar = np.array([ np.linalg.norm(end_xyz[0:2]) , end_xyz[2]])
            link_planar = np.array([np.linalg.norm(link_xyz[0:2]), link_xyz[2]])

            target_end = target_planar-end_planar
            target_link = target_planar-link_planar
            end_link = end_planar-link_planar

            direction = np.cross( end_link, target_link )

            q_delta = solve_cosine( np.linalg.norm(target_link), np.linalg.norm(end_link), np.linalg.norm(target_end) )[0]
            if direction > 0:
                q_delta = -q_delta
            if l <=2:
                q_delta = -q_delta
            
            q[l] += q_delta        
    
            if q_limits is not None:
#                if q[l] < q_limits[l,0] or q[l] > q_limits[l,1]:
#                    q[l] = q0[l]
                q[l] = np.clip(q[l], q_limits[l,0] , q_limits[l,1])

            if verbose:
                print('Target: {}'.format(target_planar))
                print('End: {}'.format(end_planar))
                print('LINK: {}'.format(link_planar))
                print('Delta (target-end): {}'.format(target_end))
                print('Delta (target-Link): {}'.format(target_link))
                print('Delta (end-Link): {}'.format(end_link))
                print('Q_delta: {}'.format(180*q_delta/np.pi))

                plt.plot( end_planar[0], end_planar[1], 'rx' )
                plt.plot( target_planar[0], target_planar[1], 'go' )
                plt.plot( link_planar[0], link_planar[1], 'gx' )
                plt.grid()
                plt.axis('equal')
                plt.show()
    
    if verbose:
        
        
        end_xyz = robot.endpoint(q[0], q[1], q[2], q[3])[0:3,-1]
        RMSE = np.linalg.norm( target-end_xyz )
    print('RMSE: {}'.format(RMSE))
    
    return q

In [ ]:
q = [np.pi/2, 4.5*np.pi/5, np.pi/3, 0]
robot.set_angles_rads(q[0], q[1], q[2], q[3])

In [ ]:
def orient_func(q):
    orient = robot.endpoint(q[0], q[1], q[2], q[3])[0:3,-1]-robot.endpoint(q[0], q[1], q[2], q[3],0,4)[0:3,-1]
    return orient
    
def orient_J(q, eps=0.1):
    d_orient_q0 = (orient_func(q + np.array([eps,0,0,0])) - orient_func(q))/eps
    d_orient_q1 = (orient_func(q + np.array([0,eps,0,0])) - orient_func(q))/eps
    d_orient_q2 = (orient_func(q + np.array([0,0,eps,0])) - orient_func(q))/eps
    d_orient_q3 = (orient_func(q + np.array([0,0,0,eps])) - orient_func(q))/eps
    return np.stack([d_orient_q0, d_orient_q1, d_orient_q2, d_orient_q3], axis=0).transpose()
    
orient = orient_func(q)
orient_J(q)
orient_res = np.array([0.0,0.0,-82.94]) - orient
J = orient_J(q)
c = np.dot(np.dot(np.linalg.inv( np.dot(J.transpose(), J) + np.eye(4) ), J.transpose()), orient_res)
print(c)

In [ ]:

    
robot.home()
angle = 0
q0 = [np.pi/2, 4.5*np.pi/5, np.pi/3, 0]
q = q0
robot.set_angles_rads(q0[0], q0[1], q0[2], q0[3])
it = 1

q_limits = np.array([ [0,np.pi], [0,4*np.pi/5], [-np.pi,np.pi/2], [-np.pi/2,np.pi] ])


print(180*np.array(q)/np.pi)

x = 30
y = 90
z = 70

it = 0
while 1:
    
    y = 50 + 30*np.cos(angle)
    z = 50 + 30*np.sin(angle)
    x = 50 #+ 30*np.sin(angle)
    
    target = [x,y,z]


    q = solve_num(target, q=q, q_limits=q_limits, l=100, rmse_th=1e-4,  maxit=1500,step_size=1.2, epsilon=np.pi/1800)
#    q = solve_ccd(target, q=q, q_limits=q_limits, maxit=100)
    
    if 0:
        for i in range(0,10):
            orient = orient_func(q)
            orient_J(q)
            orient_res = np.array([0.0,0.0,-82.94]) - orient
            J = orient_J(q)
            c = np.dot(np.dot(np.linalg.inv( np.dot(J.transpose(), J) + np.eye(4) ), J.transpose()), orient_res)
            q += c
            for l in range(0,4):
                q[l] = np.clip(q[l], q_limits[l,0] , q_limits[l,1])
        q = solve_num(target, q=q, q_limits=q_limits, l=100, rmse_th=1e-4,  maxit=1500,step_size=1.2, epsilon=np.pi/1800)
        print('Orientation: {}\nresidual:{}'.format(orient, orient_res))
    
    robot.set_angles_rads(q[0], q[1], q[2], q[3])
    
    #draw_situation(q[0], q[1], q[2], q[3])

    
#    if it % 3 == 0:
#        clear_output()
    #print(180*q/np.pi)

    sleep(0.001)
    angle += np.pi/30
    x = -x
    it +=1


In [ ]:
q[1] = 0
while 1:
    
    robot.set_angles_rads(q[0], q[1], q[2], q[3])
    q[1] += 0.01
    sleep(0.1)

In [ ]:
robot.home()
def ik(x,y,z):
    q = [np.pi/2, np.pi/2, np.pi/2, 0]
    
    q[0] = math.atan2(y,x)
    
    q[3] =  math.atan2(  z, math.sqrt(x*x+y*y) ) -  math.atan2(83, 35)
    print(q[0])
    robot.set_angles_rads(q[0], q[1], q[2], q[3])
    

ik(50, 0, 120)
    